# Energy Loss WML Deployment Recipe

This notebook demonstrates the use of AI Model Factory recipes to deploy energy loss pipeline to WML

## Import libraries

In [ ]:
import json
import requests
import yaml

## Set Variables

model_uri - URI to fetch the model from \
deployment_name - name to set for the deployment

In [ ]:
deployment_name = "energyloss_111523_v2_three_kpis_loss_rate_joe"

In [ ]:
model_factory_config_file_name = "../config/model_info_three_kpis_loss_rate.yml"


with open(model_factory_config_file_name, 'r') as file:
    model_factory_config = yaml.safe_load(file)

print(model_factory_config)

In [ ]:
model_uri = model_factory_config['s3_uri']

In [ ]:
print(model_uri)

In [ ]:
endpoint_url = "http://localhost:8000/ibm/modelfactory/service"
credentials_path = "../../config/credentials/wml-credentials.json"

In [ ]:
url = endpoint_url + "/deployment/wml/create"

## Create a request to call WML deployment recipe

In [ ]:
payload = {
    'endpoint': 'electrical_transformer_deployments',
    'deployment_name': deployment_name,
    'model_uri': model_uri,
    'flavor': 'sklearn',
    'software_spec_name': 'modelfactory',
    
}

In [ ]:
files={
      "wml_credentials_file": ('wml-credentials.json',open(credentials_path,'rb'),'application/json')
}

In [ ]:
headers = {
  'accept': 'application/json'
}

In [ ]:
response = requests.request("POST", url, headers=headers, data=payload, files=files)

In [ ]:
print(response.text)

## Check the status of the job

In [ ]:
job_id = json.loads(response.text)["job_id"]

In [ ]:
url = endpoint_url + "/log" + f"/{job_id}"

In [ ]:
response = requests.request("GET", url, headers=headers, data={}, files=[])

In [ ]:
print(json.loads(response.text)["logs"])

## Get the summary of the job

In [ ]:
url = endpoint_url + "/summary" + f"/{job_id}"

In [ ]:
response = requests.request("GET", url, headers=headers, data={}, files=[])

In [ ]:
# More print functions using display 
from IPython.display import display, HTML
import requests

def print_job_details(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    summary_data = get_response.json()
    
    # Display the job status
    display(HTML(print_keys_and_values(summary_data)))

    
def print_keys_and_values(json_data):
    # Start the HTML code
    html_code = "<div style='font-family: Arial; font-size: 1.2em;'>"
    
    # Add the job details to the HTML code
    html_code += f"<p>job_id: {json_data['job_id']}</p>"
    if "status" in json_data:
        html_code += f"<p>status: {json_data['status']}</p>"
    html_code += "<br>"
    if "detailed_summary" in json_data:
        for summary in json_data['detailed_summary']:
            html_code += f"<p>run_id: {summary['run_id']}</p>"
            html_code += f"<p>experiment_id: {summary['experiment_id']}</p>"
            html_code += f"<p>status: {summary['status']}</p>"
            html_code += f"<p>artifact_uri: {summary['artifact_uri']}</p>"
            html_code += f"<p>artifact_name: {summary.get('tags.artifact_name', 'No artifact_name found')}</p>"
            html_code += "<br>"
    
    # Close the HTML code
    html_code += "</div>"
    
    return html_code

In [ ]:
print_job_details(job_id, endpoint_url)

In [ ]:
# Check the summary
summary_url = endpoint_url + "/summary/"
get_response = requests.get(summary_url+job_id, headers={})
summary_data = get_response.json()
print(summary_data)

In [ ]:
summary_data

In [ ]:
deployment_endpoint = summary_data['summary']['deployment_details']['entity']['status']

In [ ]:
deployment_endpoint

In [ ]:
deployment_endpoint['serving_urls']

In [ ]:
deployment_endpoint['cp4d_url'] = deployment_endpoint['serving_urls'][0]

In [ ]:
deployment_endpoint['cp4d_url']

In [ ]:
from datetime import datetime
# Get the current date
current_date = datetime.now()

# Format the date and create the string
version_string = f"?version={current_date.strftime('%Y-%m-%d')}"

In [ ]:
deployment_endpoint['cp4d_url'] = deployment_endpoint['cp4d_url'] + version_string

In [ ]:
deployment_endpoint

In [ ]:
with open("../config/deployment_info_three_kpis_loss_rate.yml","w") as file:
    yaml.dump(deployment_endpoint, file)